# Enter State Farm

In [1]:
from theano.sandbox import cuda
cuda.use('gpu0')

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29



In [2]:
%matplotlib inline
from __future__ import print_function, division
path = "data/state/"
#path = "data/state/sample/"
import utils; reload(utils)
from utils import *
from IPython.display import FileLink

Using Theano backend.


In [3]:
batch_size=64

## Setup batches

In [4]:
batches = get_batches(path+'train', batch_size=batch_size)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

Found 17252 images belonging to 10 classes.
Found 5172 images belonging to 10 classes.


In [5]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 17252 images belonging to 10 classes.
Found 5172 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


Rather than using batches, we could just import all the data into an array to save some processing time. (In most examples I'm using the batches, however - just because that's how I happened to start out.)

In [6]:
trn = get_data(path+'train')
val = get_data(path+'valid')

Found 17252 images belonging to 10 classes.


KeyboardInterrupt: 

In [7]:
save_array(path+'results/val.dat', val)
save_array(path+'results/trn.dat', trn)

In [7]:
val = load_array(path+'results/val.dat')
trn = load_array(path+'results/trn.dat')

## Re-run sample experiments on full dataset

We should find that everything that worked on the sample (see statefarm-sample.ipynb), works on the full dataset too. Only better! Because now we have more data. So let's see how they go - the models in this section are exact copies of the sample notebook models.

### Single conv layer

In [9]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample,verbose=2)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample,verbose=2)
    return model

In [10]:
model = conv1(batches)

Epoch 1/2
313s - loss: 0.2237 - acc: 0.9421 - val_loss: 2.3188 - val_acc: 0.3968
Epoch 2/2
292s - loss: 0.0123 - acc: 0.9988 - val_loss: 1.5964 - val_acc: 0.5224
Epoch 1/4
300s - loss: 0.0045 - acc: 0.9997 - val_loss: 1.5684 - val_acc: 0.5683
Epoch 2/4
297s - loss: 0.0020 - acc: 0.9999 - val_loss: 1.4986 - val_acc: 0.5839
Epoch 3/4
297s - loss: 0.0012 - acc: 1.0000 - val_loss: 1.5358 - val_acc: 0.5907
Epoch 4/4
298s - loss: 8.9791e-04 - acc: 1.0000 - val_loss: 1.5854 - val_acc: 0.5791


Interestingly, with no regularization or augmentation we're getting some reasonable results from our simple convolutional model. So with augmentation, we hopefully will see some very good results.

### Data augmentation

In [11]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 17252 images belonging to 10 classes.


In [12]:
model = conv1(batches)

Epoch 1/2
311s - loss: 1.2722 - acc: 0.5893 - val_loss: 2.1296 - val_acc: 0.3826
Epoch 2/2
301s - loss: 0.6369 - acc: 0.8012 - val_loss: 1.5866 - val_acc: 0.5159
Epoch 1/4
313s - loss: 0.4337 - acc: 0.8709 - val_loss: 1.5413 - val_acc: 0.5431
Epoch 2/4
301s - loss: 0.3315 - acc: 0.9064 - val_loss: 1.4896 - val_acc: 0.6087
Epoch 3/4
304s - loss: 0.2612 - acc: 0.9309 - val_loss: 1.5820 - val_acc: 0.5435
Epoch 4/4
302s - loss: 0.2149 - acc: 0.9412 - val_loss: 1.3618 - val_acc: 0.5683


In [13]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=15, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/15
310s - loss: 0.1929 - acc: 0.9481 - val_loss: 1.4152 - val_acc: 0.5967
Epoch 2/15
302s - loss: 0.1706 - acc: 0.9555 - val_loss: 1.4667 - val_acc: 0.6025
Epoch 3/15
302s - loss: 0.1485 - acc: 0.9628 - val_loss: 1.6622 - val_acc: 0.5319
Epoch 4/15
306s - loss: 0.1314 - acc: 0.9663 - val_loss: 1.6869 - val_acc: 0.5810
Epoch 5/15
308s - loss: 0.1228 - acc: 0.9679 - val_loss: 1.4759 - val_acc: 0.6255
Epoch 6/15
308s - loss: 0.1144 - acc: 0.9692 - val_loss: 1.5043 - val_acc: 0.5665
Epoch 7/15
301s - loss: 0.1052 - acc: 0.9730 - val_loss: 1.5910 - val_acc: 0.5683
Epoch 8/15
301s - loss: 0.1031 - acc: 0.9726 - val_loss: 1.5107 - val_acc: 0.5949
Epoch 9/15
300s - loss: 0.0944 - acc: 0.9758 - val_loss: 1.5428 - val_acc: 0.6046
Epoch 10/15
300s - loss: 0.0860 - acc: 0.9779 - val_loss: 1.5229 - val_acc: 0.6332
Epoch 11/15
305s - loss: 0.0808 - acc: 0.9804 - val_loss: 1.4394 - val_acc: 0.6156
Epoch 12/15
301s - loss: 0.0804 - acc: 0.9786 - val_loss: 1.5085 - val_acc: 0.5793
Epoch 13/15
3

I'm shocked by *how* good these results are! We're regularly seeing 75-80% accuracy on the validation set, which puts us into the top third or better of the competition. With such a simple model and no dropout or semi-supervised learning, this really speaks to the power of this approach to data augmentation.

### Four conv/pooling pairs + dropout

Unfortunately, the results are still very unstable - the validation accuracy jumps from epoch to epoch. Perhaps a deeper model with some dropout would help.

In [8]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(path+'train', gen_t, batch_size=batch_size)

Found 17252 images belonging to 10 classes.


In [9]:
model = Sequential([
        BatchNormalization(axis=1, input_shape=(3,224,224)),
        Convolution2D(32,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(64,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(128,3,3, activation='relu'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Flatten(),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(200, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

In [10]:
model.compile(Adam(lr=10e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/2
347s - loss: 2.5612 - acc: 0.2685 - val_loss: 2.2873 - val_acc: 0.1618
Epoch 2/2
301s - loss: 1.6847 - acc: 0.4625 - val_loss: 1.6332 - val_acc: 0.4971


In [12]:
model.optimizer.lr=0.001

In [13]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/10
303s - loss: 1.2383 - acc: 0.5904 - val_loss: 1.5688 - val_acc: 0.5532
Epoch 2/10
296s - loss: 0.9324 - acc: 0.6922 - val_loss: 1.3636 - val_acc: 0.5889
Epoch 3/10
298s - loss: 0.7627 - acc: 0.7508 - val_loss: 1.3769 - val_acc: 0.6060
Epoch 4/10
294s - loss: 0.6066 - acc: 0.7975 - val_loss: 1.6376 - val_acc: 0.5538
Epoch 5/10
294s - loss: 0.5107 - acc: 0.8318 - val_loss: 1.2229 - val_acc: 0.6520
Epoch 6/10
298s - loss: 0.4325 - acc: 0.8597 - val_loss: 1.0864 - val_acc: 0.6794
Epoch 7/10
298s - loss: 0.3802 - acc: 0.8772 - val_loss: 1.0607 - val_acc: 0.6810
Epoch 8/10
301s - loss: 0.3438 - acc: 0.8881 - val_loss: 1.2468 - val_acc: 0.6382
Epoch 9/10
296s - loss: 0.3072 - acc: 0.9009 - val_loss: 1.1046 - val_acc: 0.6661
Epoch 10/10
297s - loss: 0.2798 - acc: 0.9089 - val_loss: 1.0775 - val_acc: 0.6877


In [14]:
model.optimizer.lr=0.00001

In [15]:
model.fit_generator(batches, batches.nb_sample, nb_epoch=10, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample, verbose=2)

Epoch 1/10
303s - loss: 0.2506 - acc: 0.9206 - val_loss: 1.2009 - val_acc: 0.6808
Epoch 2/10
296s - loss: 0.2283 - acc: 0.9295 - val_loss: 1.1299 - val_acc: 0.6541
Epoch 3/10
296s - loss: 0.2114 - acc: 0.9332 - val_loss: 1.4639 - val_acc: 0.6346
Epoch 4/10
301s - loss: 0.2026 - acc: 0.9368 - val_loss: 1.1740 - val_acc: 0.6916
Epoch 5/10
293s - loss: 0.1868 - acc: 0.9413 - val_loss: 1.1446 - val_acc: 0.6920
Epoch 6/10
301s - loss: 0.1810 - acc: 0.9428 - val_loss: 1.1244 - val_acc: 0.7131
Epoch 7/10
295s - loss: 0.1751 - acc: 0.9448 - val_loss: 1.1762 - val_acc: 0.7026
Epoch 8/10
296s - loss: 0.1565 - acc: 0.9514 - val_loss: 1.3924 - val_acc: 0.6750
Epoch 9/10
299s - loss: 0.1512 - acc: 0.9526 - val_loss: 1.0721 - val_acc: 0.7251
Epoch 10/10
299s - loss: 0.1445 - acc: 0.9532 - val_loss: 1.2523 - val_acc: 0.6841


This is looking quite a bit better - the accuracy is similar, but the stability is higher. There's still some way to go however...

### Imagenet conv features

Since we have so little data, and it is similar to imagenet images (full color photos), using pre-trained VGG weights is likely to be helpful - in fact it seems likely that we won't need to fine-tune the convolutional layer weights much, if at all. So we can pre-compute the output of the last convolutional layer, as we did in lesson 3 when we experimented with dropout. (However this means that we can't use full data augmentation, since we can't pre-compute something that changes every image.)

In [16]:
vgg = Vgg16()
model=vgg.model
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [17]:
conv_model = Sequential(conv_layers)

In [18]:
# batches shuffle must be set to False when pre-computing features
batches = get_batches(path+'train', batch_size=batch_size, shuffle=False)
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(path+'test', batch_size=batch_size*2, shuffle=False)

Found 17252 images belonging to 10 classes.
Found 5172 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [19]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 17252 images belonging to 10 classes.
Found 5172 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [20]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [21]:
save_array(path+'results/conv_val_feat.dat', conv_val_feat)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)
save_array(path+'results/conv_feat.dat', conv_feat)

In [22]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_val_feat.shape

(5172, 512, 14, 14)

### Batchnorm dense layers on pretrained conv layers

Since we've pre-computed the output of the last convolutional layer, we need to create a network that takes that as input, and predicts our 10 classes. Let's try using a simplified version of VGG's dense layers.

In [23]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [24]:
p=0.8

In [25]:
bn_model = Sequential(get_bn_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [27]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels), verbose=2)

INFO (theano.gof.compilelock): Refreshing lock /home/ubuntu/.theano/compiledir_Linux-4.4--aws-x86_64-with-debian-stretch-sid-x86_64-2.7.13-64/lock_dir/lock
Problem occurred during compilation with the command line below:
/usr/bin/g++ -shared -g -O3 -fno-math-errno -Wno-unused-label -Wno-unused-variable -Wno-write-strings -march=broadwell -mmmx -mno-3dnow -msse -msse2 -msse3 -mssse3 -mno-sse4a -mcx16 -msahf -mmovbe -maes -mno-sha -mpclmul -mpopcnt -mabm -mno-lwp -mfma -mno-fma4 -mno-xop -mbmi -mbmi2 -mno-tbm -mavx -mavx2 -msse4.2 -msse4.1 -mlzcnt -mrtm -mhle -mrdrnd -mf16c -mfsgsbase -mno-rdseed -mno-prfchw -mno-adx -mfxsr -mxsave -mxsaveopt -mno-avx512f -mno-avx512er -mno-avx512cd -mno-avx512pf -mno-prefetchwt1 -mno-clflushopt -mno-xsavec -mno-xsaves -mno-avx512dq -mno-avx512bw -mno-avx512vl -mno-avx512ifma -mno-avx512vbmi -mno-clwb -mno-pcommit -mno-mwaitx --param l1-cache-size=32 --param l1-cache-line-size=64 --param l2-cache-size=46080 -mtune=broadwell -DNPY_NO_DEPRECATED_API=NPY_1_

OSError: [Errno 12] Cannot allocate memory

In [ ]:
bn_model.optimizer.lr=0.01

In [ ]:
bn_model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels), verbose=2)

In [ ]:
bn_model.save_weights(path+'models/conv8.h5')

Looking good! Let's try pre-computing 5 epochs worth of augmented data, so we can experiment with combining dropout and augmentation on the pre-trained model.

### Pre-computed data augmentation + dropout

We'll use our usual data augmentation parameters:

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
da_batches = get_batches(path+'train', gen_t, batch_size=batch_size, shuffle=False)

We use those to create a dataset of convolutional features 5x bigger than the training set.

In [ ]:
da_conv_feat = conv_model.predict_generator(da_batches, da_batches.nb_sample*5)

In [ ]:
save_array(path+'results/da_conv_feat2.dat', da_conv_feat)

In [ ]:
da_conv_feat = load_array(path+'results/da_conv_feat2.dat')

Let's include the real training data as well in its non-augmented form.

In [ ]:
da_conv_feat = np.concatenate([da_conv_feat, conv_feat])

Since we've now got a dataset 6x bigger than before, we'll need to copy our labels 6 times too.

In [ ]:
da_trn_labels = np.concatenate([trn_labels]*6)

Based on some experiments the previous model works well, with bigger dense layers.

In [ ]:
def get_bn_da_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]

In [ ]:
p=0.8

In [ ]:
bn_model = Sequential(get_bn_da_layers(p))
bn_model.compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Now we can train the model as usual, with pre-computed augmented data.

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels), verbose=2)

In [ ]:
bn_model.optimizer.lr=0.01

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels),verbose=2)

In [ ]:
bn_model.optimizer.lr=0.0001

In [ ]:
bn_model.fit(da_conv_feat, da_trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels),verbose=2)

Looks good - let's save those weights.

In [ ]:
bn_model.save_weights(path+'models/da_conv8_1.h5')

### Pseudo labeling

We're going to try using a combination of [pseudo labeling](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf) and [knowledge distillation](https://arxiv.org/abs/1503.02531) to allow us to use unlabeled data (i.e. do semi-supervised learning). For our initial experiment we'll use the validation set as the unlabeled data, so that we can see that it is working without using the test set. At a later date we'll try using the test set.

To do this, we simply calculate the predictions of our model...

In [ ]:
val_pseudo = bn_model.predict(conv_val_feat, batch_size=batch_size)

...concatenate them with our training labels...

In [ ]:
comb_pseudo = np.concatenate([da_trn_labels, val_pseudo])

In [ ]:
comb_feat = np.concatenate([da_conv_feat, conv_val_feat])

...and fine-tune our model using that data.

In [ ]:
bn_model.load_weights(path+'models/da_conv8_1.h5')

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels), verbose=2)

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels), verbose=2)

In [ ]:
bn_model.optimizer.lr=0.00001

In [ ]:
bn_model.fit(comb_feat, comb_pseudo, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels), verbose=2)

That's a distinct improvement - even although the validation set isn't very big. This looks encouraging for when we try this on the test set.

In [ ]:
bn_model.save_weights(path+'models/bn-ps8.h5')

### Submit

We'll find a good clipping amount using the validation set, prior to submitting.

In [ ]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, do_clip(val_preds, 0.93)).eval()

In [ ]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [ ]:
preds = bn_model.predict(conv_test_feat, batch_size=batch_size*2)

In [ ]:
subm = do_clip(preds,0.93)

In [ ]:
subm_name = path+'results/subm.gz'

In [ ]:
classes = sorted(batches.class_indices, key=batches.class_indices.get)

In [ ]:
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'img', [a[4:] for a in test_filenames])
submission.head()

In [ ]:
submission.to_csv(subm_name, index=False, compression='gzip')

In [ ]:
FileLink(subm_name)

This gets 0.534 on the leaderboard.

## The "things that didn't really work" section

You can safely ignore everything from here on, because they didn't really help.

### Finetune some conv layers too

In [ ]:
for l in get_bn_layers(p): conv_model.add(l)

In [ ]:
for l1,l2 in zip(bn_model.layers, conv_model.layers[last_conv_idx+1:]):
    l2.set_weights(l1.get_weights())

In [ ]:
for l in conv_model.layers: l.trainable =False

In [ ]:
for l in conv_model.layers[last_conv_idx+1:]: l.trainable =True

In [ ]:
comb = np.concatenate([trn, val])

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=8, height_shift_range=0.04, 
                shear_range=0.03, channel_shift_range=10, width_shift_range=0.08)

In [ ]:
batches = gen_t.flow(comb, comb_pseudo, batch_size=batch_size)

In [ ]:
val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)

In [ ]:
conv_model.compile(Adam(lr=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=1, validation_data=val_batches, 
                 nb_val_samples=val_batches.N, verbose=2)

In [ ]:
conv_model.optimizer.lr = 0.0001

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=3, validation_data=val_batches, 
                 nb_val_samples=val_batches.N, verbose=2)

In [ ]:
for l in conv_model.layers[16:]: l.trainable =True

In [ ]:
conv_model.optimizer.lr = 0.00001

In [ ]:
conv_model.fit_generator(batches, batches.N, nb_epoch=8, validation_data=val_batches, 
                 nb_val_samples=val_batches.N, verbose=2)

In [ ]:
conv_model.save_weights(path+'models/conv8_ps.h5')

In [ ]:
conv_model.load_weights(path+'models/conv8_da.h5')

In [ ]:
val_pseudo = conv_model.predict(val, batch_size=batch_size*2)

In [ ]:
save_array(path+'models/pseudo8_da.dat', val_pseudo)

### Ensembling

In [ ]:
drivers_ds = pd.read_csv(path+'driver_imgs_list.csv')
drivers_ds.head()

In [ ]:
img2driver = drivers_ds.set_index('img')['subject'].to_dict()

In [ ]:
driver2imgs = {k: g["img"].tolist() 
               for k,g in drivers_ds[['subject', 'img']].groupby("subject")}

In [ ]:
def get_idx(driver_list):
    return [i for i,f in enumerate(filenames) if img2driver[f[3:]] in driver_list]

In [ ]:
drivers = driver2imgs.keys()

In [ ]:
rnd_drivers = np.random.permutation(drivers)

In [ ]:
ds1 = rnd_drivers[:len(rnd_drivers)//2]
ds2 = rnd_drivers[len(rnd_drivers)//2:]

In [ ]:
models=[fit_conv([d]) for d in drivers]
models=[m for m in models if m is not None]

In [ ]:
all_preds = np.stack([m.predict(conv_test_feat, batch_size=128) for m in models])
avg_preds = all_preds.mean(axis=0)
avg_preds = avg_preds/np.expand_dims(avg_preds.sum(axis=1), 1)

In [ ]:
keras.metrics.categorical_crossentropy(val_labels, np.clip(avg_val_preds,0.01,0.99)).eval()

In [ ]:
keras.metrics.categorical_accuracy(val_labels, np.clip(avg_val_preds,0.01,0.99)).eval()